In [1]:
from qiskit import *
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
import matplotlib.pyplot as plotter
import numpy as np
from IPython.display import display, Math, Latex
%matplotlib inline
# Loading your IBM Q account(s)

from qiskit.pulse.pulse_lib import Gaussian
import matplotlib.pyplot as plt
import qiskit.pulse as pulse
import qiskit.pulse.library as pulse_lib
from qiskit.compiler import assemble
from qiskit.pulse.library import SamplePulse

from qiskit.tools.monitor import job_monitor


<ipython-input-1-027f393ed2f2>:11: DeprecationWarning: the pulse_lib module is deprecated, pulse_lib is renamed to library
  from qiskit.pulse.pulse_lib import Gaussian


In [2]:
pi_01_amp = 0.22320005035637547
freq_01 = 4.994453427300261
drive_pow = 0.3
drive_sigma_us = 0.075                     
drive_samples_us = drive_sigma_us*8

def get_closest_multiple_of_16(num):
    
    return (int(num) - (int(num)%16))

In [3]:
import warnings
warnings.filterwarnings('ignore')
from qiskit.tools.jupyter import *
%matplotlib inline

from qiskit import IBMQ
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
backend = provider.get_backend('ibmq_armonk')

backend_config = backend.configuration()
assert backend_config.open_pulse, "Backend doesn't support Pulse"

dt = backend_config.dt

backend_defaults = backend.defaults()

# unit conversion factors -> all backend properties returned in SI (Hz, sec, etc)
GHz = 1.0e9 # Gigahertz
MHz = 1.0e6 # Megahertz
us = 1.0e-6 # Microseconds
ns = 1.0e-9 # Nanoseconds


In [4]:
drive_sigma = get_closest_multiple_of_16(drive_sigma_us * us /dt)       
drive_samples = get_closest_multiple_of_16(drive_samples_us * us /dt)

In [5]:
pi_pulse_01 = pulse_lib.gaussian(duration=drive_samples,
                                 amp=pi_01_amp, 
                                 sigma=drive_sigma,
                                 name='pi_pulse_01')

cal_qubit_freq = 4.974489

In [6]:
def apply_sideband(pulse, freq):
    # time goes from 0 to dt*drive_samples, sine arg of form 2*pi*f*t
    t_samples = np.linspace(0, dt*drive_samples, drive_samples)
    sine_pulse = np.sin(2*np.pi*(freq-cal_qubit_freq)*t_samples) # no amp for the sine
    
    # create sample pulse w/ sideband applied
    # Note: need to make sq_pulse.samples real, multiply elementwise
    sideband_pulse = SamplePulse(np.multiply(np.real(pulse.samples), sine_pulse), name='sideband_pulse')
    
    return sideband_pulse    




qubit_freq_12 = 4.6263002
pi_amp_12 = 0.37256049920143336

pi_pulse_12 = pulse_lib.gaussian(duration=drive_samples,
                                 amp=pi_amp_12, 
                                 sigma=drive_sigma,
                                 name='pi_pulse_12')
# make sure this pulse is sidebanded
pi_pulse_12 = apply_sideband(pi_pulse_12, qubit_freq_12)

In [7]:
meas_map_idx = None
for i, measure_group in enumerate(backend_config.meas_map):
    if qubit in measure_group:
        meas_map_idx = i
        break
assert meas_map_idx is not None, f"Couldn't find qubit {qubit} in the meas_map!"

NameError: name 'qubit' is not defined

In [ ]:
#Here we have calibrated the pi_pulse_01 and pi_pulse_12

#For making a bit flip from 0 -> 2
#We will apply a pi_pulse of 0 -> 1 and then 1 -> 2
#Similarly for 2 -> 0 we will just use our pi_pulses in reverse order
qubit = 0

inst_sched_map = backend_defaults.instruction_schedule_map
measure = inst_sched_map.get('measure', qubits=backend_config.meas_map[meas_map_idx])


drive_chan = pulse.DriveChannel(qubit)
meas_chan = pulse.MeasureChannel(qubit)
acq_chan = pulse.AcquireChannel(qubit)


two_schedule = pulse.Schedule(name="two schedule")
two_schedule |= pulse.Play(pi_pulse_01, drive_chan)
two_schedule |= pulse.Play(pi_pulse_12, drive_chan) << two_schedule.duration
two_schedule |= measure << two_schedule.duration